# the cryptopals crypto challenges

## Crypto Challenge Set 1

Cryptopals Rule:
>Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing.

### Encodings

- ASCII (256). [Table](https://www.ascii-code.com/compact)
- HEX (16)

### Numbering Systems

1. Binary (base 2)
2. [Octal](https://www.electronics-tutorials.ws/binary/bin_4.html) (base 8)
3. Decimal (base 10)
4. Hexadecimal (base 16)

| Binary | 0 | 1 | 10 | 11 | 100 | 101 | 110 | 111 | 1000 | 1001 | 1010 | 1011 | 1100 | 1101 | 1110 | 1111 | 10000 |
| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:| -:|
| Decimal | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9  | 10 | 11 | 12 | 13 | 14 | 15 | 16 |
| Hexadecimal | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9  | A | B | C | D | E | F | 10 |

### Example

Represent the decimal number 512 in hex: `512 = 2 x 16^2 + 0 x 16^1 + 0 x 160 = 200`

### Challenge 1 - Convert hex to base64

In [1]:
from enum import Enum

class Mode(Enum):
    ENCRYPT = "ENCRYPT"
    DECRYPT = "DECRYPT"


In [2]:
import base64
from typing import Union

def convert_hex_to_base64(hex_str: str) -> Union[bytes, None]:
  try:
    # first decode string from hex representation
    raw_bytes = bytearray.fromhex(hex_str)
    # now encode bytes to base64 encoding
    b64_bytes = base64.b64encode(raw_bytes)
    return b64_bytes
  except ValueError:
    print(f"Invalid hexadecimal string: {hex_str}")
    return None


In [3]:
# https://docs.python.org/3/library/stdtypes.html#bytes.decode
assert convert_hex_to_base64("49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d").decode() == "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"
print(f'Set 1 Challenge 1 passing')

Set 1 Challenge 1 passing


### Challenge 2 - Fixed XOR

>In cryptography, the simple XOR cipher is a type of additive cipher, an encryption algorithm that operates according to the principles:

```
A ^ 0 = A
A ^ A = 0
A ^ B = B ^ A
(A ^ B) ^ C = A ^ (B ^ C)
(B ^ A) ^ A = B ^ 0 = B
```

>where `^` denotes the exclusive disjunction (XOR) operation. This operation is sometimes called modulus 2 addition (or subtraction, which is identical). With this logic, a string of text can be encrypted by applying the bitwise XOR operator to every character using a given key. To decrypt the output, merely reapplying the XOR function with the key will remove the cipher.

*[Source](https://en.wikipedia.org/wiki/XOR_cipher)*

#### XOR Cipher Trace Table

| Plaintext | Key | Ciphertext |
| - | - | - |
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

In [4]:
# def xor(bytes1: bytearray, bytes2: bytearray) -> Union[bytes, None]:
#   if len(bytes1) != len(bytes2):
#     print("Input byte arrays must be of the same length.")
#     return None
  
#   return bytes([b1 ^ b2 for b1, b2 in zip(bytes1, bytes2)])

def xor(bytes1: bytearray, bytes2: bytearray) -> bytes:
  return bytes([b1 ^ b2 for b1, b2 in zip(bytes1, bytes2)])

In [5]:
assert xor(bytearray.fromhex("1c0111001f010100061a024b53535009181c"), bytearray.fromhex("686974207468652062756c6c277320657965")).hex() == "746865206b696420646f6e277420706c6179"
print(f'Set 1 Challenge 2 passing')

Set 1 Challenge 2 passing


### Challenge 3 - Single-byte XOR cipher

In [6]:
from typing import Dict
from string import ascii_lowercase
from collections.abc import Iterable

def character_frequency_score(input_bytes: bytearray) -> int:
  letter_frequency = { 'e': 12.70, 't': 9.05, 'a': 8.16, 'o': 7.50, 'i': 6.96, 'n': 6.74, 's': 6.32, 'h': 6.09, 'r': 5.98, 'd': 4.25, 'l': 4.02, 'c': 2.78, 'u': 2.75, 'm': 2.40, 'w': 2.36, 'f': 2.22, 'g': 2.01, 'y': 1.97, 'p': 1.92, 'b': 1.49, 'v': 0.97, 'k': 0.77, 'j': 0.15, 'x': 0.15, 'q': 0.09, 'z': 0.07 }
  allowed_characters = ['.', ',', '\'', '"', ' ', '!', '?', '-']

  score = 0
  for c in input_bytes:
    if c < 0 or c > 127:
      return -1000
    c = chr(c)
    if not c.isalnum() and c not in allowed_characters:
        score -= 50
    elif c.isalpha():
        score += letter_frequency[c.lower()] * 10

  return score

def single_byte_xor_cipher(cipher_str: bytearray) -> Dict[str, Union[bytearray, int]]:
  result = None

  for n in range(0, 256):
    # When using a single character/int for key, just duplicate to the length of the cipher
    key = n.to_bytes(1, byteorder='big')
    keystream = key * len(cipher_str)
    xored_bytes = xor(cipher_str, keystream)
    score = character_frequency_score(xored_bytes)
    if result == None or result["score"] < score: 
      result = {"plaintext": xored_bytes, "key": n, "score": score}

  return result

cipher = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"
# print(single_byte_xor_cipher(cipher))
cipher = bytearray.fromhex(cipher) # hex decode
keystream = (88).to_bytes(1, "big") * len(cipher)
xor(cipher, keystream)

b"Cooking MC's like a pound of bacon"

In [7]:
assert single_byte_xor_cipher(bytearray.fromhex("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"))["key"] == 88
print(f'Set 1 Challenge 3 passing')

Set 1 Challenge 3 passing


### Challenge 4 - Detect single-character XOR

In [8]:
def detect_single_character_xor() -> Union[str, Union[bytearray, int]]:
  best_result = None
  with open('4.txt', 'r') as file:
      lines = file.readlines()

  for line in lines:
      line = bytearray.fromhex(line.strip())  # convert line to bytearray
      result = single_byte_xor_cipher(line.strip())
      if best_result is None or best_result["score"] < result["score"]:
          best_result = result

  return best_result

detect_single_character_xor()

{'plaintext': b'Now that the party is jumping\n', 'key': 53, 'score': 1350.8}

In [9]:
assert detect_single_character_xor()["key"] == 53
print(f'Set 1 Challenge 4 passing')

Set 1 Challenge 4 passing


### Challenge 5 - Implement repeating-key XOR

In [10]:
# TODO: Type Annotations
import binascii

def apply_repeating_key_xor(input_bytes: bytearray, key_bytes: bytearray) -> str:
    encrypted_bytes = bytearray()
    key_length = len(key_bytes)

    for i, byte in enumerate(input_bytes):
        # Rotate through the key's characters
        key_byte = key_bytes[i % key_length]
        encrypted_bytes.append(byte ^ key_byte)

    encrypted_text_hex = binascii.hexlify(encrypted_bytes).decode('ascii')
    return encrypted_text_hex

# cipher_array = repeating_key_xor("Burning 'em, if you ain't quick and nimble", "ICE")
# binascii.hexlify(cipher_array).decode('ascii')

In [11]:
input5 = """Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal"""
output5 = """0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"""

input_bytes = bytearray(input5, 'utf-8')
key_bytes = bytearray("ICE", 'utf-8')

assert apply_repeating_key_xor(input_bytes, key_bytes) == output5
print(f'Set 1 Challenge 5 passing')

Set 1 Challenge 5 passing


### Challenge 6 - Break repeating-key XOR

Challenge 6 involves breaking a repeating-key XOR cipher. This type of cipher works by taking a plaintext message and a key, and then XORing the plaintext with the key repeated over and over again. If the key is shorter than the plaintext, it is repeated as many times as necessary.

The challenge is to decrypt a message that has been encrypted with this type of cipher, without knowing the key. The key insight is that the key size can be guessed by looking at the Hamming distance between different parts of the ciphertext.

The Hamming distance between two strings of equal length is the number of positions at which the corresponding symbols are different. In the context of this challenge, it is used to measure how different two byte sequences are.

The idea is that if you guess the key size correctly, then the Hamming distance between two blocks of ciphertext that were encrypted with the same part of the key should be relatively low, because the same key was used to encrypt both blocks. On the other hand, if you guess the key size incorrectly, then the Hamming distance between two blocks of ciphertext should be relatively high, because different parts of the key were used to encrypt the blocks.

So, the strategy is to guess different key sizes, calculate the average Hamming distance between blocks of ciphertext for each guessed key size, and then choose the key size with the lowest average Hamming distance.

Once the key size is known, the ciphertext can be broken into blocks of that size, and each block can be decrypted independently, because each block was encrypted with the same part of the key. This reduces the problem to breaking a single-character XOR cipher, which can be done using frequency analysis.

In summary, the Hamming distance is used to guess the key size, and then the key size is used to break the ciphertext into blocks that can be decrypted independently.

In [12]:
from typing import List

def hamming_distance(bytes1: bytearray, bytes2: bytearray) -> int:
  """
  Calculate the Hamming distance between two byte arrays.

  Parameters:
  bytes1, bytes2 (bytearray): The byte arrays to compare.

  Returns:
  int: The Hamming distance between the byte arrays.
  """
  distance = 0
  for b1, b2 in zip(bytes1, bytes2):
    diff = bin(b1 ^ b2)
    count = diff.count('1') # a 1 denoting a difference, see above table of XOR
    distance += count

  return distance

assert hamming_distance(b'''this is a test''', b'''wokka wokka!!!''') == 37

def to_blocks(lst: List[int], n: int) -> List[List[int]]:
  """
  Split a list into blocks of a given size.

  Parameters:
  lst (List[int]): The list to split.
  n (int): The size of the blocks.

  Returns:
  List[List[int]]: The list split into blocks.
  """
  for i in range(0, len(lst), n):
      yield lst[i:i + n]

assert list(to_blocks([1,2,3,4,5,6,7,8], 2)) == [[1,2],[3,4],[5,6],[7,8]]
assert list(to_blocks([1,2,3,4,5,6,7,8], 3)) == [[1,2,3],[4,5,6],[7,8]]
assert list(to_blocks([], 4)) == []

def find_keysize(ciphertext: bytearray, min_length: int = 2, max_length: int = 40) -> List[Dict[str, Union[float, int]]]:
  """
  Find the most likely key size for a repeating-key XOR cipher.

  Parameters:
  ciphertext (bytearray): The encrypted text.
  min_length, max_length (int): The minimum and maximum key sizes to consider.

  Returns:
  List[Dict[str, Union[float, int]]]: A list of dictionaries containing the average Hamming distance and the corresponding key size for each key size considered, sorted by average Hamming distance.
  """
  keysize_scores = []
  for KEYSIZE in range(min_length, max_length):
    slice_size = 2 * KEYSIZE
    # // floor division 100 // 3 = 33
    measurements = len(ciphertext) // slice_size - 1
    average_distance = 0
    for i in range(measurements):
      first = slice(i * slice_size, i * slice_size + KEYSIZE)
      second = slice(i * slice_size + KEYSIZE, i * slice_size + 2 * KEYSIZE)

      average_distance += hamming_distance(ciphertext[first], ciphertext[second])
    average_distance /= KEYSIZE
    average_distance /= measurements
    keysize_scores.append({ "average_distance": average_distance, "keysize": KEYSIZE})

  lowest_hamming = sorted(keysize_scores, key = lambda d: d["average_distance"])

  return lowest_hamming

def break_repeating_key_xor(ciphertext: bytearray) -> bytearray:
  """
  Decrypt a repeating-key XOR cipher.

  Parameters:
  ciphertext (bytearray): The encrypted text.

  Returns:
  bytearray: The decrypted text.
  """
  KEYSIZE = find_keysize(ciphertext)[0]["keysize"]
  # Try the best three keys
  # for KEYSIZE in KEYSIZES[:3]:
  key = bytes()
  plaintext = []
  for i in range(KEYSIZE):
    chunk = single_byte_xor_cipher(bytes(ciphertext[i::KEYSIZE]))
    k = bytes(chunk["key"])
    plaintext.append(chunk["plaintext"])
    key += k

  message = bytes()
  for i in range(max(map(len, plaintext))):
    message += bytes([chunk[i] for chunk in plaintext if len(chunk) >= i + 1])

  return message

In [13]:
file6 = open('6.txt', 'r')
ciphertext6 = base64.b64decode(file6.read())

assert find_keysize(ciphertext6)[0]["keysize"] == 29
assert break_repeating_key_xor(ciphertext6).decode("utf-8").startswith("I'm back and I'm ringin'")

print("Set 1 Challenge 6 passing")
# print(break_repeating_key_xor(ciphertext6))

Set 1 Challenge 6 passing


### Challenge 7 - AES in ECB mode

AES is a block cipher. One key feature of block ciphers is they can only encrypt a specific sized block of plaintext. To encrypt plaintext that is not exactly 128 bits, it needs to use *padding* and a *mode of operation*.

In [14]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

def aes_ecb_decrypt(data: bytes, key: bytes) -> bytes:
  """
  Decrypt data using AES in ECB mode.

  Parameters:
  data, key (bytes): The data to decrypt and the key to use for the decryption.

  Returns:
  bytes: The decrypted data.
  """
  if not isinstance(data, bytes) or not isinstance(key, bytes):
      raise TypeError("Data and key must be bytes.")
  if len(data) % len(key) != 0:
      raise ValueError("Data length must be a multiple of key length.")

  cipher = Cipher(algorithms.AES(key), modes.ECB())
  decryptor = cipher.decryptor()

  return decryptor.update(data) + decryptor.finalize()

def aes_ecb_encrypt(data: bytes, key: bytes) -> bytes:
  """
  Encrypt data using AES in ECB mode.

  Parameters:
  data, key (bytes): The data to encrypt and the key to use for the encryption.

  Returns:
  bytes: The encrypted data.
  """
  if not isinstance(data, bytes) or not isinstance(key, bytes):
      raise TypeError("Data and key must be bytes.")
  if len(data) % len(key) != 0:
      raise ValueError("Data length must be a multiple of key length.")

  cipher = Cipher(algorithms.AES(key), modes.ECB())
  encryptor = cipher.encryptor()

  return encryptor.update(data) + encryptor.finalize()

In [15]:
input7 = b'aaaarandom datat'
key = b'YELLOW SUBMARINE'
assert aes_ecb_decrypt(aes_ecb_encrypt(input7, key), key) == input7

In [16]:
with open('7.txt', 'r') as input_file:
    ciphertext7 = base64.b64decode(input_file.read())

key = b'YELLOW SUBMARINE'

# Check that the decrypted text starts with the expected string
assert aes_ecb_decrypt(ciphertext7, key).decode("utf-8").startswith("I'm back and I'm ringin'")

print("Set 1 Challenge 7 passing")

Set 1 Challenge 7 passing


### Challenge 8 - Detect AES in ECB mode

>In this file are a bunch of hex-encoded ciphertexts.
>
>One of them has been encrypted with ECB.
>
>Detect it.
>
>Remember that the problem with ECB is that it is stateless and deterministic; the same 16 byte plaintext block will always produce the same 16 byte ciphertext.

Electronic codebook (ECB) is the mode of operation that divides the message into blocks of 16 bytes and pads the last block if it's not 16 bytes long.
Encryption is deterministic, encrypting the same block twice leads to the same ciphertext. Also, this means encrypting block by block the resulting ciphertext might have repeating patterns.

In [17]:
def detect_aes_in_ecb_mode(file_path: str) -> Dict[str, Union[int, bytearray]]:
  """
  Detect AES in ECB mode in a file by looking for duplicate blocks in the ciphertext.

  Parameters:
  file_path (str): The path to the file.

  Returns:
  dict: A dictionary containing the count of duplicate blocks and the line with the most duplicate blocks.
  """
  with open(file_path, 'r') as file:
      lines = file.readlines()

  duplicate_counts = []
  for line in lines:
      cipher = bytearray.fromhex(line.strip())
      # split each line into 16 byte long chunks and count duplicates
      blocks = list(to_blocks(cipher, 16))
      hex_blocks = [binascii.hexlify(block) for block in blocks]
      duplicate_counts.append({ "sum": sum(hex_blocks.count(block) for block in hex_blocks), "line": cipher })

  max_item = max(duplicate_counts, key=lambda x:x["sum"])
  return max_item


In [18]:
assert detect_aes_in_ecb_mode('8.txt')["sum"] == 22
print("Set 1 Challenge 8 passing")

Set 1 Challenge 8 passing


## Crypto Challenge Set 2

### Challenge 9 - Implement PKCS#7 padding

>A block cipher transforms a fixed-sized block (usually 8 or 16 bytes) of plaintext into ciphertext. But we almost never want to transform a single block; we encrypt irregularly-sized messages.
>
>One way we account for irregularly-sized messages is by padding, creating a plaintext that is an even multiple of the blocksize. The most popular padding scheme is called PKCS#7.
>
>So: pad any block to a specific block length, by appending the number of bytes of padding to the end of the block. For instance,
>
>```
>"YELLOW SUBMARINE"
>```
>... padded to 20 bytes would be:
>
>```
>"YELLOW SUBMARINE\x04\x04\x04\x04"
>```

In [19]:
import binascii

def pkcs7_padding(text, block_size):
  padding_size = block_size - len(text) % block_size
  return text + chr(padding_size).encode('ascii') * padding_size

# def pkcs7_padding(data: bytes, block_size: int) -> bytes:
  # if len(data) >= block_size: return data
  # padding_size = block_size - len(data) % block_size
  # padding size needed to determine what int we pad with
  # 1 bit missing  -> pad with \x01
  # 4 bits missing -> pad with \x04
  # return data.ljust(block_size, chr(padding_size).encode('ascii'))
  # return text + chr(padding_size).encode('ascii') * padding_size

assert pkcs7_padding(b'YELLOW SUBMARINE', 20) == b'YELLOW SUBMARINE\x04\x04\x04\x04'
print("Set 2 Challenge 1 passing")

Set 2 Challenge 1 passing


In [20]:
def has_padding(data: bytes) -> bool:
  # grab number on padding
  # grab end to number
  padding = data[-data[-1]:]

  return all(padding[bit] == len(padding) for bit in range(0, len(padding)))

assert has_padding(b'YELLOW SUBMARINE\x04\x04\x04\x04') == True

def remove_padding(data: bytes) -> bytes:
  if has_padding(data):
    padding_length = data[len(data) - 1]
    return data[:-padding_length]
  
  return data

assert remove_padding(b'YELLOW SUBMARINE\x04\x04\x04\x04') == b'YELLOW SUBMARINE'

### Challenge 10 - Implement CBC mode

>CBC mode is a block cipher mode that allows us to encrypt irregularly-sized messages, despite the fact that a block cipher natively only transforms individual blocks.
>
>In CBC mode, each ciphertext block is added to the next plaintext block before the next call to the cipher core.
>
>The first plaintext block, which has no associated previous ciphertext block, is added to a "fake 0th ciphertext block" called the initialization vector, or IV.
>
>Implement CBC mode by hand by taking the ECB function you wrote earlier, making it encrypt instead of decrypt (verify this by decrypting whatever you encrypt to test), and using your XOR function from the previous exercise to combine them.
>
>The file here is intelligible (somewhat) when CBC decrypted against "YELLOW SUBMARINE" with an IV of all ASCII 0 (\x00\x00\x00 &c)

*Cipher block chaining* (CBC) uses an *initialization vector* (IC) to randomize the encryption. IV has the length of the block size and must be random and unpredicatble.

1. Generate random, unpredictable IV of blocksize length
2. XOR IV and first block
3. Encrypt the XORed result
4. XOR the just produced ciphertext with the next block of plaintext
5. Encrypt plaintext 
6. If there's more plaintext, go to 4

In [21]:
import os

# def aes_ecb_encrypt(data, key):
#   cipher = Cipher(algorithms.AES128(key), modes.ECB())
#   encryptor = cipher.encryptor()

#   return encryptor.update(pkcs7_padding(data, len(key))) + encryptor.finalize()

def aes_cbc_decrypt(ciphertext: bytearray, key: bytearray, iv: bytearray) -> bytearray:
  blocks = list(to_blocks(ciphertext, 16))
  plaintext = bytes()
  for block in blocks:
    # if len(b) != 16: b = pkcs7_padding(b, 16)
    plaintext += xor(aes_ecb_decrypt(block, key), iv)
    iv = block

  return remove_padding(plaintext)


def aes_cbc_encrypt(plaintext: bytearray, key: bytearray, iv: bytearray) -> bytearray:
  IV = iv
  blocks = list(to_blocks(plaintext, 16))
  ciphertext = bytes()
  for block in blocks:
    if len(block) != 16: 
        block = pkcs7_padding(block, 16)
    xored = xor(block, IV)
    encrypted = aes_ecb_encrypt(xored, key)
    ciphertext += encrypted
    IV = encrypted  # Update IV to the last block of the ciphertext

  return ciphertext

  # IV = iv
  # blocks = list(to_blocks(plaintext, 16))
  # ciphertext = bytes()
  # for i, block in enumerate(blocks):
  #   if len(block) != 16 and i == len(blocks) - 1:
  #     block = pkcs7_padding(block, 16)
  #   xored = xor(block, IV)
  #   ciphertext += aes_ecb_encrypt(xored, key)
  #   IV = ciphertext

  # return ciphertext

# def aes_cbc_encrypt(plaintext: bytearray, key: bytearray, iv: bytearray) -> bytearray:
#   IV = iv
#   blocks = list(to_blocks(plaintext, 16))
#   ciphertext = bytes()
#   for block in blocks:
#     if len(block) != 16: block = pkcs7_padding(block, 16)
#     xored = xor(block, IV)
#     ciphertext += aes_ecb_encrypt(xored, key)
#     IV = ciphertext

#   return ciphertext



In [22]:
STARTING_IV = os.urandom(16)
input10 = b'This is not 16 bytes'
key = b'YELLOW SUBMARINE'
# cipher = aes_cbc_encrypt(input10, key, STARTING_IV)
# plain = aes_cbc_decrypt(cipher, key, STARTING_IV)
# print(cipher)
# print(plain)
assert aes_cbc_decrypt(aes_cbc_encrypt(input10, key, STARTING_IV), key, STARTING_IV) == input10
print('Set 2 Challenge 10 passing')
# aes_in_ecb_mode(cipher, b'YELLOW SUBMARINE')

Set 2 Challenge 10 passing


### Challenge 11 - An ECB/CBC detection oracle

>Now that you have ECB and CBC working:
>
>Write a function to generate a random AES key; that's just 16 random bytes.
>
>Write a function that encrypts data under an unknown key --- that is, a function that generates a random key and encrypts under it.
>
>The function should look like:
>
```
encryption_oracle(your-input)
=> [MEANINGLESS JIBBER JABBER]
```
>Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.
>
>Now, have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). Use rand(2) to decide which to use.
>
>Detect the block cipher mode the function is using each time. You should end up with a piece of code that, pointed at a block box that might be encrypting ECB or CBC, tells you which one is happening.

In [35]:
import random
import secrets

def aes_key(length = 16):
  return os.urandom(length)

assert len(aes_key()) == 16
assert aes_key() != aes_key()

def aes_ecb_encrypt(data, key):
  cipher = Cipher(algorithms.AES(key), modes.ECB())
  encryptor = cipher.encryptor()

  return encryptor.update(pkcs7_padding(data, 16)) + encryptor.finalize()

def encryption_oracle(data):
    # 1. append 5-10 bytes (randomly) before and after plaintext
    # 2. pick randomly (rand(2)) to encrypt using ECB or CBC
    # 3. use aes_key function 
    number_of_bytes = random.randint(5, 10)
    random_bytes = os.urandom(number_of_bytes)
    padded_data = pkcs7_padding(random_bytes + data + random_bytes, 16)
    encrypt_mode = random.randint(1, 2)

    match encrypt_mode:
       case 1:
          print('Encrypting with CBC')
          STARTING_IV = os.urandom(16)
          return aes_cbc_encrypt(padded_data, aes_key(), STARTING_IV)
       case 2:
          print('Encrypting with ECB')
          return aes_ecb_encrypt(padded_data, aes_key())

def detect_block_cipher(ciphertext: bytes) -> str:
    """
    Detect the block cipher mode (ECB or CBC) used to encrypt a ciphertext.

    Parameters:
    ciphertext (bytes): The ciphertext to analyze.

    Returns:
    str: The detected block cipher mode ("ECB" or "CBC").
    """
    # Split the ciphertext into blocks
    blocks = [ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)]

    # Count the number of unique blocks
    unique_blocks = len(set(blocks))

    # If the number of unique blocks is less than the total number of blocks,
    # then there are repeated blocks in the ciphertext, which suggests that
    # ECB mode was used. Otherwise, CBC mode was likely used.
    return "ECB" if unique_blocks < len(blocks) else "CBC"

print(f"Detected {detect_block_cipher(encryption_oracle(b'This is 16 bytes'))}")

Encrypting with CBC
Detected ECB


In [24]:
# puts data into chunks of length (16)
s = b'123456789abcdefghijkl'
[s[i:i+16] for i in range(0, len(s), 16)]

[b'123456789abcdefg', b'hijkl']